In [1]:
import doctest
import io
import re
from dataclasses import dataclass
from typing import List, Optional, Tuple

In [2]:
DATA = "input.txt"

# Part 1

Naive solution is to expand all the ranges and insert the mappings into dictionaries. Given this is AoC it's likely the ranges are going to get large so assuming that's a bad idea! 

Idea: The ranges are non-overlapping so store the source range starts in a list, sorted by source start, along with the range length and destination range starts. To map a source integer to a destination integer we search for the largest value less than or equal to the source integer, check if it is within the range and then map it to the destination range.

In [3]:
class Map:
    """

    Args:

        data: List of ``(dst start, src start, rng len)``.

    Example:

        >>> m = Map([(50, 98, 2), (52, 50, 48)])
        >>> m[98]
        50
        >>> m[99]
        51
        >>> m[100]
        100
        >>> m[60]
        62

        >>> m = Map([(0, 15, 37), (37, 52, 2), (39, 0, 15)])
        >>> m[0]
        39
        >>> m[14]
        53
        >>> m[53]
        38
        >>> m[15]
        0
    """
    def __init__(self, data: List[Tuple[int, int, int]]):
        self._data = sorted(data, key=lambda x: x[1])

    def __repr__(self) -> str:
        return f"Map(data={self._data})"

    def __getitem__(self, src: int) -> Optional[int]:
        # find range
        start, end = 0, len(self._data) - 1
        while True:
            i = start + ((end - start) // 2)
            entry = self._data[i]

            if entry[1] <= src and src < entry[1] + entry[2]:
                return entry[0] + (src - entry[1])

            if src < entry[1]:
                end = i - 1

            if src >= entry[1] + entry[2]:
                start = i + 1

            if start > end or end < start:
                break
        return src

In [4]:
def parse_input(data: io.TextIOBase) -> Tuple[List[int], List[Map]]:
    """Returns the list of seeds to be planted and the list of maps.

    Example:

        >>> data = io.StringIO('''seeds: 79 14 55 13
        ... 
        ... seed-to-soil map:
        ... 50 98 2
        ... 52 50 48
        ... ''')
        >>> parse_input(data)
        ([79, 14, 55, 13], [Map(data=[(52, 50, 48), (50, 98, 2)])])
    """
    # assumes well-formed input, no error handling!
    data_iter = iter(data)
    seeds = [int(s) for s in next(data_iter)[7:].split()]
    maps = []

    next(data_iter) # blank line
    while True:
        try:
            map_name = next(data_iter)
        except StopIteration:
            break
        map_entries = []
        while True:
            try:
                map_entry = next(data_iter)
            except StopIteration:
                break
            if map_entry == "\n":
                break
            map_entries.append(tuple(int(me) for me in map_entry.split()))
        maps.append(Map(map_entries))
            
    return seeds, maps

In [5]:
def map_to_locations(seeds: List[int], maps: List[Map]) -> List[int]:
    """Returns the seeds mapped to the locations.

    Example:

        >>> data = io.StringIO('''seeds: 79 14 55 13
        ...
        ... seed-to-soil map:
        ... 50 98 2
        ... 52 50 48
        ...
        ... soil-to-fertilizer map:
        ... 0 15 37
        ... 37 52 2
        ... 39 0 15
        ...
        ... fertilizer-to-water map:
        ... 49 53 8
        ... 0 11 42
        ... 42 0 7
        ... 57 7 4
        ...
        ... water-to-light map:
        ... 88 18 7
        ... 18 25 70
        ...
        ... light-to-temperature map:
        ... 45 77 23
        ... 81 45 19
        ... 68 64 13
        ...
        ... temperature-to-humidity map:
        ... 0 69 1
        ... 1 0 69
        ...
        ... humidity-to-location map:
        ... 60 56 37
        ... 56 93 4
        ... ''')
        >>> seeds, maps = parse_input(data)
        >>> map_to_locations(seeds, maps)
        [82, 43, 86, 35]
    """
    locations = []
    for seed in seeds:
        n = seed
        for i, m in enumerate(maps):
            n = m[n]
        locations.append(n)
    return locations

In [6]:
doctest.testmod()

TestResults(failed=0, attempted=15)

In [7]:
with open(DATA, "r") as f:
    seeds, maps = parse_input(f)
print(min(map_to_locations(seeds, maps)))

178159714


# Part 2

Idea: Each initial `(start, length)` pair becomes a set of intervals initialised as `{(src start, src start + length)}`. Mapping maps these intervals to `(dst start, length)` pairs. However, the interval may span more than one range so the mapping may split one "source" interval into one or more destination interval hence the need for a set. The mapping process continues until all location intervals are retrieved, at which point the lowest `src start` can be selected. 

The `Map.__getitem__` method needs updating to return multiple ranges given an input range.

In [171]:
class IntervalMap(Map):
    def __getitem__(self, src: Tuple[int, int]) -> set[Tuple[int, int]]:
        """

        Example:

            >>> m = IntervalMap([(50, 98, 2), (52, 50, 48)])
            >>> m[(90, 10)]
            [(92, 8), (50, 2)]

        """
        src_start, src_length = src
        
        # find interval that `src_start` falls in
        start, end = 0, len(self._data) - 1
        while True:
            i = start + ((end - start) // 2)
            entry = self._data[i]

            if entry[1] <= src_start and src_start < entry[1] + entry[2]:
                break

            if src_start < entry[1]:
                end = i - 1

            if src_start >= entry[1] + entry[2]:
                start = i + 1

            if start > end or end < start:
                return {src}

        # entry contains src_start, now scan right producing the new interval(s)
        new_intervals = []
        cur_pos = src_start
        remain_length = src_length
        while cur_pos < src_start + src_length:
            if i >= len(self._data):
                new_intervals.append((cur_pos, remain_length))
                cur_pos += remain_length
                break
            entry = self._data[i]
            if entry[1] + entry[2] >= cur_pos + remain_length:
                new_intervals.append((entry[0] + (cur_pos - entry[1]), remain_length))
                cur_pos += remain_length
            else:
                interval_len = entry[1] + entry[2] - cur_pos
                new_intervals.append((entry[0] + (cur_pos - entry[1]), interval_len))
                cur_pos += interval_len
                remain_length -= interval_len
                i += 1
        
        return new_intervals

In [172]:
def map_intervals_to_locations(intervals: List[Tuple[int, int]], maps: List[Map]) -> List[Tuple[int, int]]:
    """Returns the seed intervals mapped to location intervals.

    Example:

        >>> data = io.StringIO('''seeds: 82 1
        ...
        ... seed-to-soil map:
        ... 50 98 2
        ... 52 50 48
        ...
        ... soil-to-fertilizer map:
        ... 0 15 37
        ... 37 52 2
        ... 39 0 15
        ...
        ... fertilizer-to-water map:
        ... 49 53 8
        ... 0 11 42
        ... 42 0 7
        ... 57 7 4
        ...
        ... water-to-light map:
        ... 88 18 7
        ... 18 25 70
        ...
        ... light-to-temperature map:
        ... 45 77 23
        ... 81 45 19
        ... 68 64 13
        ...
        ... temperature-to-humidity map:
        ... 0 69 1
        ... 1 0 69
        ...
        ... humidity-to-location map:
        ... 60 56 37
        ... 56 93 4
        ... ''')
        >>> seeds, maps = parse_input(data)
        >>> intervals = list(zip(seeds[::2], seeds[1::2]))
        >>> maps = [IntervalMap(m._data) for m in maps]
        >>> map_intervals_to_locations(intervals, maps)
        [{(46, 1)}]
    """
    locations = []
    for seed_interval in intervals:
        ints = {seed_interval}
        for i, m in enumerate(maps):
            new_ints = set()
            for interval in ints:
                new_int = m[interval]
                new_ints.update(new_int)
            ints = new_ints
        locations.append(ints)
    return locations

In [173]:
doctest.testmod()

TestResults(failed=0, attempted=22)

In [174]:
with open(DATA, "r") as f:
    seeds, maps = parse_input(f)
intervals = list(zip(seeds[::2], seeds[1::2]))
maps = [IntervalMap(m._data) for m in maps]
locations = map_intervals_to_locations(intervals, maps)
min([l for intervals in locations for l, _ in intervals])

100165128